<a href="https://colab.research.google.com/github/Elvirasun28/anomality_detection/blob/master/Low_Pass_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os

path = "/content/gdrive/My Drive/Colab Notebooks/anomality_detection"
os.chdir(path)
os.listdir(path)

In [0]:
from __future__ import division
from itertools import count
import matplotlib.pyplot as plt 
from numpy import linspace, loadtxt, ones, convolve
import numpy as np
import pandas as pd
import collections
from random import randint 
from matplotlib import style 
style.use('fivethirtyeight')

In [0]:
## load dataset 
data = loadtxt('data/sunspots.txt',np.float)
data_as_frame = pd.DataFrame(data,columns=['Months','SunSpots'])

In [0]:
def moving_average(data,window_size):
    window = np.ones(int(window_size)) / np.float(window_size)
    return np.convolve(data,window,'same')

def explain_anomalies(y,window_size, sigma=1.0):
    ''' exploring the anamolies using stationary standard deviation '''
    avg = moving_average(y, window_size),tolist()
    residual = y - avg 
    std = np.std(residual)
    return {'standard_deviation':round(std,3),
            'anomalies_dict':collections.OrderedDict([(idx,y_i) for idx, y_i, avg_i in zip(count(),y,avg)
                    if (y_i > avg_i + (sigma * std)) | (y_i < avg_i - (sigma*std))
                    ])
            
            }


def explain_anomalies_rolling_std(y,window_size, sigma = 1.0):
    ''' exploring the anamolies using rolling standard deviation '''
    avg = moving_average(y,window_size).tolist()
    residual = y - avg 
    # caculate the variation in the distribution of the residual 
    testing_std = pd.rolling_std(residual,window_size)
    testing_std_as_df = pd.DataFrame(testing_std)
    rolling_std = testing_std_as_df.replace(np.nan,
                                  testing_std_as_df.ix[window_size - 1]).round(3).iloc[:,0].tolist()
    